In [1]:
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

# Data Preprocessing

In [2]:
df = pd.read_csv('data/winequality-red.csv')

Missing values (NaN)

In [3]:
for i, feat in enumerate(df.columns):
    df.dropna(subset = [feat], inplace=True)

Outliers removal

In [4]:
import scipy.stats as stats
z_scores = stats.zscore(df)

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df = df[filtered_entries]

Build input and target

In [5]:
X = df.drop("quality",1)    #Feature Matrix
Y = df["quality"]           #Target Variable

In [6]:
X.corrwith(Y)

fixed acidity           0.145163
volatile acidity       -0.353443
citric acid             0.243999
residual sugar          0.061482
chlorides              -0.108787
free sulfur dioxide    -0.071202
total sulfur dioxide   -0.237745
density                -0.167568
pH                     -0.082164
sulphates               0.386567
alcohol                 0.501501
dtype: float64

Balance of influence

In [7]:
###################
# INSERT CODE HERE
###################
## TODO: Balance the influence of features by scaling them to similar distributions
## /!\ make sure you end with pandas X as a pandas.Dataframe
##
## X = ?
features = X.columns

scaler = StandardScaler()
X[features] = scaler.fit_transform(X[features])

In [8]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.552930,1.050914,-1.390400,-0.565439,-0.258851,-0.441060,-0.330784,0.635485,1.375896,-0.636458,-0.999856
1,-0.309900,2.119336,-1.390400,0.246832,0.790825,1.063255,0.790477,0.052390,-0.821951,0.288353,-0.608174
2,-0.309900,1.407054,-1.180831,-0.101284,0.504550,-0.011256,0.348768,0.169009,-0.396561,0.057150,-0.608174
3,1.755851,-1.442071,1.543569,-0.565439,-0.306564,0.203646,0.552634,0.752103,-1.105544,-0.482323,-0.608174
4,-0.552930,1.050914,-1.390400,-0.565439,-0.258851,-0.441060,-0.330784,0.635485,1.375896,-0.636458,-0.999856
...,...,...,...,...,...,...,...,...,...,...,...
1594,-1.282019,0.457346,-0.971261,-0.449401,0.409125,1.815413,0.008992,-1.055489,0.950506,-0.482323,0.077270
1595,-1.464291,0.160562,-0.866477,-0.217323,-0.926828,2.567570,0.246835,-0.927209,1.446794,0.904894,0.762714
1596,-1.221261,-0.076865,-0.709300,-0.101284,-0.258851,1.493059,-0.126918,-0.565690,0.737811,0.827826,0.566873
1597,-1.464291,0.724451,-0.761692,-0.449401,-0.306564,1.815413,0.008992,-0.723125,1.801286,0.519556,-0.216491


Train Test Split

In [9]:
###################
# INSERT CODE HERE
###################
## TODO: Split your dataset in two parts: train split and test split.
## X_train, X_test, y_train, y_test = ?
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

In [10]:
print(f'Training Samples shape: {X_train.shape} ; Training Target shape: {y_train.shape}\nTest Samples shape: {X_test.shape}; Test Target shape {y_test.shape}')

Training Samples shape: (1305, 11) ; Training Target shape: (1305,)
Test Samples shape: (146, 11); Test Target shape (146,)


# Metrics

In [11]:
# Compute the Root Mean Square Error
def compute_rmse(predict, target):
    ###################
    # INSERT CODE HERE
    ###################
    ## TODO: PUT your implementation of RMSE  from previous sessions
    ## return rmse
    if len(target.shape) == 2:
        target = target.squeeze()
    if len(predict.shape) == 2:
        predict = predict.squeeze()
    diff = target - predict
    if len(diff.shape) == 1:
        diff = np.expand_dims(diff, axis=-1)
    rmse = np.sqrt(diff.T@diff / diff.shape[0])
    return float(rmse)


# Feature Selection

In [12]:
def select_features(X_train, Y_train, X_test, n_feat):
    ###################
    # INSERT CODE HERE
    ###################
    ## TODO: Return X_train_filtered, X_test_filtered such that they only contain the n_feats
    ##       that are the most correlated to the target
    cor = np.abs(X_train.corrwith(Y_train))
    most_correlated_features = cor.sort_values(ascending=False)[:n_feat]
    bests_features = [feature for feature, _ in most_correlated_features.items()]
    
    return X_train[bests_features], X_test[bests_features]

In [28]:
# select_features(X_train, y_train, X_test, 4)

type(X_train)


pandas.core.frame.DataFrame

# K-Fold Cross Validation

In [13]:
###################
# INSERT CODE HERE
###################
## TODO: Instanciate  a K-fold object from sklearn such that k = 5 
## kf = 
kf = KFold(n_splits=5, shuffle=True)

In [14]:
def run_kfold(kf, X_train_filtered, y_train, n_features):
    ###################
    # INSERT CODE HERE
    ###################
    ## TODO: Implement K-fold Cross Validation for a linear regressor and 
    ##       print the mean and std of the rmse of this cross-validation
    ##
    rmse = []
    
    for train_index, test_index in kf.split(X_train_filtered):

        X_train, X_test = X_train_filtered.values[train_index], X_train_filtered.values[test_index]
        y_train_, y_test = y_train.values[train_index], y_train.values[test_index]

        lin_reg = LinearRegression()
        lin_reg.fit(X_train, y_train_)

        y_pred = lin_reg.predict(X_test)

        rmse.append(compute_rmse(y_pred, y_test))

    print('RMSE AVG:', np.mean(rmse), 'STD:', np.std(rmse))
    


In [15]:
for n_features in [2,3,4,5,6]:
    print(f'----------- n features: {n_features} ---------------------')
    X_train_filtered, X_test_filtered = select_features(X_train, y_train, X_test , n_features)
    run_kfold(kf, X_train_filtered, y_train, n_features)

----------- n features: 2 ---------------------
RMSE AVG: 0.641025166705531 STD: 0.01726262336757198
----------- n features: 3 ---------------------
RMSE AVG: 0.6272602874448842 STD: 0.041337665796106295
----------- n features: 4 ---------------------
RMSE AVG: 0.6234554301537496 STD: 0.015044350365886161
----------- n features: 5 ---------------------
RMSE AVG: 0.6249867236180029 STD: 0.021828156205156133
----------- n features: 6 ---------------------
RMSE AVG: 0.6235287889371401 STD: 0.025482288949202056


# Grid Search

In [16]:
from sklearn.svm import SVR
from sklearn.metrics import make_scorer

def perform_grid_search(model_, hyper_params_grid, score_function):
    ###################
    # INSERT CODE HERE
    ###################
    ## TODO: 1) define model_ function as model selection criterion using 'make_scorer'
    ##       2) Perform a grid search, train your models on the training set
    ##       3) Print the values of the hyperparameters used by the best model
    ##       4) return your grid search object
    scorer = make_scorer(score_function)
    grid = GridSearchCV(model_, hyper_params_grid, scoring=scorer)
    print(grid)
    return grid
    

###################
# INSERT CODE HERE
###################
## TODO: 1) Define the model to use to perform a grid search -> support vector regressor
##       2) define a hyperparameter grid on 'kernel', 'degree', 'gamma'  -> see SVR documentation
##       3) call perform_grid_search to train SVR models on the trining data 
##       4)  predict the testing data with the best model using the best parameter
model = SVR()

hyperparameter = {
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree' : [1, 2, 3, 5, 10],
    'gamma'  : ['scale', 'auto']
}

grid = perform_grid_search(model, hyperparameter, lambda x, y : - compute_rmse(x, y))

grid.fit(X_train.values, y_train.values)
grid_predictions = grid.predict(X_test.values)

print('RMSE:', compute_rmse(grid_predictions, y_test.to_numpy()))

GridSearchCV(estimator=SVR(),
             param_grid={'degree': [1, 2, 3, 5, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring=make_scorer(<lambda>))
RMSE: 0.5884550111486009


In [17]:
# see the report of grid search
res = pd.DataFrame(grid.cv_results_.values(), ).transpose()
res.columns=grid.cv_results_.keys()
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_degree,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.082249,0.009091,0.00254,0.000197,1,scale,linear,"{'degree': 1, 'gamma': 'scale', 'kernel': 'lin...",-0.627113,-0.634862,-0.649927,-0.583417,-0.632502,-0.625564,0.022391,11
1,0.027972,0.001331,0.002508,0.00007,1,scale,poly,"{'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}",-0.627085,-0.635032,-0.650251,-0.583188,-0.632623,-0.625636,0.022569,22
2,0.032198,0.002279,0.008189,0.001174,1,scale,rbf,"{'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}",-0.605885,-0.6217,-0.609124,-0.555763,-0.623938,-0.603282,0.024758,6
3,0.055793,0.003623,0.009557,0.000837,1,scale,sigmoid,"{'degree': 1, 'gamma': 'scale', 'kernel': 'sig...",-8.058184,-9.106303,-8.693906,-8.070131,-8.543754,-8.494455,0.396725,29
4,0.088707,0.008104,0.003011,0.000397,1,auto,linear,"{'degree': 1, 'gamma': 'auto', 'kernel': 'line...",-0.627113,-0.634862,-0.649927,-0.583417,-0.632502,-0.625564,0.022391,11
5,0.03343,0.008181,0.002994,0.000497,1,auto,poly,"{'degree': 1, 'gamma': 'auto', 'kernel': 'poly'}",-0.627112,-0.635031,-0.650243,-0.583171,-0.632621,-0.625636,0.022575,21
6,0.047835,0.002807,0.00966,0.001033,1,auto,rbf,"{'degree': 1, 'gamma': 'auto', 'kernel': 'rbf'}",-0.605534,-0.621322,-0.60911,-0.556227,-0.623941,-0.603227,0.024517,1
7,0.070209,0.007685,0.013892,0.004763,1,auto,sigmoid,"{'degree': 1, 'gamma': 'auto', 'kernel': 'sigm...",-8.226712,-9.432937,-8.69738,-8.116056,-8.552429,-8.605103,0.464531,34
8,0.103359,0.016596,0.004213,0.001772,2,scale,linear,"{'degree': 2, 'gamma': 'scale', 'kernel': 'lin...",-0.627113,-0.634862,-0.649927,-0.583417,-0.632502,-0.625564,0.022391,11
9,0.047343,0.006231,0.004497,0.001551,2,scale,poly,"{'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}",-0.72356,-0.747404,-0.781048,-0.744257,-0.705175,-0.740289,0.025458,26


In [18]:
###################
# INSERT CODE HERE
###################
## TODO: Same thing but with a decision tree regressor
from sklearn.tree import DecisionTreeRegressor

model_DT = DecisionTreeRegressor()

hyperparameter_DT = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter' : ['best', 'random'],
}

grid_DT = perform_grid_search(model_DT, hyperparameter_DT,
                           lambda x, y: - compute_rmse(x, y))

grid_DT.fit(X_train.values, y_train.values)
grid_predictions_DT = grid_DT.predict(X_test.values)

print('RMSE:', compute_rmse(grid_predictions_DT, y_test.to_numpy()))


GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'splitter': ['best', 'random']},
             scoring=make_scorer(<lambda>))
RMSE: 0.8758557459257962


In [19]:
res_DT = pd.DataFrame(grid_DT.cv_results_.values(), ).transpose()
res_DT.columns=grid_DT.cv_results_.keys()
res_DT

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004977,0.000567,0.000183,0.000037,squared_error,best,"{'criterion': 'squared_error', 'splitter': 'be...",-0.76814,-0.775585,-0.80468,-0.678064,-0.727142,-0.750722,0.043973,1
1,0.001368,0.000186,0.000138,0.000047,squared_error,random,"{'criterion': 'squared_error', 'splitter': 'ra...",-0.758098,-0.862146,-0.790266,-0.745356,-0.823505,-0.795874,0.042816,6
2,0.005384,0.000434,0.00027,0.00013,friedman_mse,best,"{'criterion': 'friedman_mse', 'splitter': 'best'}",-0.760621,-0.792687,-0.811791,-0.669534,-0.755567,-0.75804,0.048858,2
3,0.001452,0.000036,0.00013,0.000009,friedman_mse,random,"{'criterion': 'friedman_mse', 'splitter': 'ran...",-0.816497,-0.814147,-0.802296,-0.740198,-0.740198,-0.782667,0.035009,4
4,0.314274,0.056616,0.000296,0.00003,absolute_error,best,"{'criterion': 'absolute_error', 'splitter': 'b...",-0.724503,-0.84191,-0.799904,-0.785403,-0.753027,-0.780949,0.040153,3
5,0.110481,0.031423,0.000292,0.000055,absolute_error,random,"{'criterion': 'absolute_error', 'splitter': 'r...",-0.763135,-0.797506,-0.7951,-0.839631,-0.797506,-0.798576,0.024314,7
6,0.020886,0.000667,0.000218,0.00003,poisson,best,"{'criterion': 'poisson', 'splitter': 'best'}",-0.770629,-0.828145,-0.846449,-0.866578,-0.828145,-0.827989,0.032,8
7,0.002591,0.000223,0.000145,0.000027,poisson,random,"{'criterion': 'poisson', 'splitter': 'random'}",-0.763135,-0.807057,-0.823505,-0.799904,-0.773111,-0.793343,0.022193,5
